In [2]:
#Import library
import numpy as np
import pandas as pd
import vectorbt as bt
import yfinance as yf
import pandas_ta as ta

In [4]:
#Get Data
btc = yf.download('BTC-USD', start='2024-12-04', end='2024-12-11', interval='15m')
btc.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
btc = btc.asfreq('15min')
btc.tail()

[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Datetime,,,,,
2024-12-10 22:45:00,96944.890625,96944.890625,96798.007812,96798.007812,0
2024-12-10 23:00:00,97061.710938,97164.054688,96972.546875,96972.546875,0
2024-12-10 23:15:00,97045.945312,97087.039062,96899.671875,96899.671875,0
2024-12-10 23:30:00,96826.585938,97136.046875,96826.585938,97045.546875,0
2024-12-10 23:45:00,96681.914062,96732.703125,96614.171875,96732.703125,142614528


In [3]:
# Initial parameters
initial_capital = 100000
risk_per_trade = 0.02  # 2%sl_trail=True
stop_loss = 0.18  # 18%
take_profit = 0.40  # 40%

In [4]:
#Technical Indicators
btc['rsi']= ta.rsi(btc['Close'], length=14)
btc['ema_short'] = ta.ema(btc['Close'], length=20)
btc['ema_long'] = ta.ema(btc['Close'], length=50)
btc['vwap'] = ta.vwap(btc['High'], btc['Low'], btc['Close'], btc['Volume'])
bb = ta.bbands(btc['Close'], length=20, std=2)
adx_results = ta.adx(btc['High'], btc['Low'], btc['Close'], length=5)

# Assign to DataFrame
btc['adx'] = adx_results['ADX_5']  # ADX values
btc['a'] = adx_results['DMP_5']    # Positive Directional Movement
btc['b'] = adx_results['DMN_5']    # Negative Directional Movement

btc['bbl'] = bb['BBL_20_2.0']
btc['bbm'] = bb['BBM_20_2.0']
btc['bbu'] = bb['BBU_20_2.0']





In [5]:
#Entry and Exit 
btc['long_entry'] = ((((btc['Close'] - btc['Open']) < 0.38*(btc['High'] - btc['Low'])) &
                      (btc['Close'] > btc['Open']) &
                      ((btc['High'] - btc['Close']) < 0.7*(btc['Close'] - btc['Open'])))
                      ).astype(int)
btc['long_exit'] = (btc['rsi'] > 70).astype(int)

btc['short_entry'] = ((((btc['Open'] - btc['Close']) < 0.38*(btc['High'] - btc['Low'])) &
                      (btc['Close'] < btc['Open']) &
                      ((btc['Close'] - btc['Low']) < 0.7*(btc['Open'] - btc['Close'])))
                      ).astype(int)
btc['short_exit'] = (btc['rsi']<30).astype(int)

In [22]:
#Portfolio Construction
pf = bt.Portfolio.from_signals(btc['Close'], entries=btc['long_entry'], short_entries=btc['short_entry'],fees=0.001, slippage=0.001, sl_stop=0.04, sl_trail=True, upon_stop_exit='reverse', )

In [23]:
#Returns
returns = pf.total_return()
print(returns)

-0.027311303575621223


In [24]:
#Stats
pf.stats()

Start                         2024-12-04 00:00:00
End                           2024-12-10 23:45:00
Period                            7 days 00:00:00
Start Value                                 100.0
End Value                                97.26887
Total Return [%]                         -2.73113
Benchmark Return [%]                     0.669528
Max Gross Exposure [%]                      100.0
Total Fees Paid                          5.969635
Max Drawdown [%]                        12.328658
Max Drawdown Duration             5 days 01:30:00
Total Trades                                   31
Total Closed Trades                            30
Total Open Trades                               1
Open Trade PnL                           -0.11119
Win Rate [%]                            43.333333
Best Trade [%]                           3.407389
Worst Trade [%]                         -3.944679
Avg Winning Trade [%]                    1.224855
Avg Losing Trade [%]                    -1.071569


In [25]:
#Plots
pf.plot(
    subplots=['trades', 'orders', 'trade_pnl', 'cum_returns']
    )


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6ab69f57-3ed6-4363-879c-866e4024fa0f',
              'x': array([datetime.datetime(2024, 12, 4, 0, 0),
                          datetime.datetime(2024, 12, 4, 0, 15),
                          datetime.datetime(2024, 12, 4, 0, 30), ...,
                          datetime.datetime(2024, 12, 10, 23, 15),
                          datetime.datetime(2024, 12, 10, 23, 30),
                          datetime.datetime(2024, 12, 10, 23, 45)], dtype=object),
              'xaxis': 'x',
              'y': array([96038.90625  , 95989.3671875, 95739.       , ..., 97045.9453125,
                          96826.5859375, 96681.9140625]),
              'yaxis': 'y'},
             {'customdata': array([[0, 0, 0.0010333300099976842, 0.09990009990009993, 'Short'],
                              